In [1]:
import pandas as pd
import scipy
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
ratings = pd.read_csv('/Users/junjie/Documents/RecommenderSystems-master/userDatas.csv')
movies = pd.read_csv('/Users/junjie/Documents/RecommenderSystems-master/netflix_titles.csv')
ratings = pd.merge(movies, ratings).drop(['date_added', 'release_year', 'rating', 'duration'], axis=1)
#ratings = pd.merge(movies, ratings)
ratings.head()

,show_id,type,title,director,cast,country,listed_in,description,user_id,fav_genre,fav_cast,user_rating
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentaries,"As her father nears the end of his life, filmm...",u5,Documentaries,NaN,2
1,s33,TV Show,Sex Education,NaN,"Asa Butterfield, Gillian Anderson, Ncuti Gatwa...",United Kingdom,"British TV Shows, International TV Shows, TV C...",Insecure Otis has all the answers when it come...,u9,Comedies,"Joel Courtney,Stephen Jennings",5
2,s46,Movie,My Heroes Were Cowboys,Tyler Greco,NaN,NaN,Documentaries,Robin Wiltshire's painful childhood was rescue...,u5,Documentaries,NaN,5
3,s47,Movie,Safe House,Daniel Espinosa,"Denzel Washington, Ryan Reynolds, Vera Farmiga...","South Africa, United States, Japan",Action & Adventure,Young CIA operative Matt Weston must get a dan...,u3,Action & Adventure,"Chris Hemsworth,Rain,Matt Damon,Jason Statham,...",3
4,s83,TV Show,Lucifer,NaN,"Tom Ellis, Lauren German, Kevin Alejandro, D.B...",United States,"Crime TV Shows, TV Comedies, TV Dramas","Bored with being the Lord of Hell, the devil r...",u9,Comedies,"Joel Courtney,Stephen Jennings",5


In [3]:
ratings['cast'] = ratings['cast'].fillna(' ')
ratings['fav_cast'] = ratings['fav_cast'].fillna(' ')
ratings['director'] = ratings['director'].fillna(' ')
ratings['country'] = ratings['country'].fillna(' ')

In [4]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='user_rating')
print("Before: ",userRatings.shape)
#userRatings = userRatings.dropna(how='all').fillna(0)
userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)
userRatings.head(10)

Before:  (10, 58)
After:  (10, 58)


title,60 Days In,Akame ga Kill!,Attack on Titan,Avengers: Infinity War,Blade Runner: The Final Cut,Bling Empire,"Bob Ross: Happy Accidents, Betrayal & Greed",Cops and Robbers,Cosmic Sin,Creating an Army of the Dead,...,The Kissing Booth 3,The Secret Life of Pets 2,The Seven Deadly Sins,The Seventh Day,The Show Must Go On: The Queen + Adam Lambert Story,Tinker Bell and the Legend of the NeverBeast,Truth or Dare,Two Distant Strangers,Why Did You Kill Me?,Worth
user_id,,,,,,,,,,,,,,,,,,,,,
u1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,5.0
u2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
u3,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u4,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u5,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
u6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0
u7,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u8,2.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [5]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

title,60 Days In,Akame ga Kill!,Attack on Titan,Avengers: Infinity War,Blade Runner: The Final Cut,Bling Empire,"Bob Ross: Happy Accidents, Betrayal & Greed",Cops and Robbers,Cosmic Sin,Creating an Army of the Dead,...,The Kissing Booth 3,The Secret Life of Pets 2,The Seven Deadly Sins,The Seventh Day,The Show Must Go On: The Queen + Adam Lambert Story,Tinker Bell and the Legend of the NeverBeast,Truth or Dare,Two Distant Strangers,Why Did You Kill Me?,Worth
title,,,,,,,,,,,,,,,,,,,,,
60 Days In,1.000000,-0.111111,-0.111111,-0.166667,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Akame ga Kill!,-0.111111,1.000000,1.000000,-0.166667,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Attack on Titan,-0.111111,1.000000,1.000000,-0.166667,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Avengers: Infinity War,-0.166667,-0.166667,-0.166667,1.000000,-0.166667,-0.166667,-0.166667,-0.166667,0.666667,-0.166667,...,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
Blade Runner: The Final Cut,-0.111111,-0.111111,-0.111111,-0.166667,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Bling Empire,1.000000,-0.111111,-0.111111,-0.166667,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
"Bob Ross: Happy Accidents, Betrayal & Greed",-0.111111,-0.111111,-0.111111,-0.166667,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,1.000000,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111
Cops and Robbers,-0.111111,-0.111111,-0.111111,-0.166667,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,1.000000
Cosmic Sin,-0.111111,-0.111111,-0.111111,0.666667,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111


In [16]:
def standardize(row):
    new_row = (row - row.mean())/(row.max()-row.min())
    return new_row

df_temp = userRatings
df_std = df_temp.apply(standardize)

user_similarity = cosine_similarity(df_std.T)
user_similarity

array([[ 1.        , -0.11111111, -0.11111111, ..., -0.11111111,
        -0.11111111, -0.11111111],
       [-0.11111111,  1.        ,  1.        , ..., -0.11111111,
        -0.11111111, -0.11111111],
       [-0.11111111,  1.        ,  1.        , ..., -0.11111111,
        -0.11111111, -0.11111111],
       ...,
       [-0.11111111, -0.11111111, -0.11111111, ...,  1.        ,
        -0.11111111,  1.        ],
       [-0.11111111, -0.11111111, -0.11111111, ..., -0.11111111,
         1.        , -0.11111111],
       [-0.11111111, -0.11111111, -0.11111111, ...,  1.        ,
        -0.11111111,  1.        ]])

In [10]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_dist = euclidean_distances(df_std.T)
euclidean_dist

array([[0.        , 1.41421356, 1.41421356, ..., 1.41421356, 1.41421356,
        1.41421356],
       [1.41421356, 0.        , 0.        , ..., 1.41421356, 1.41421356,
        1.41421356],
       [1.41421356, 0.        , 0.        , ..., 1.41421356, 1.41421356,
        1.41421356],
       ...,
       [1.41421356, 1.41421356, 1.41421356, ..., 0.        , 1.41421356,
        0.        ],
       [1.41421356, 1.41421356, 1.41421356, ..., 1.41421356, 0.        ,
        1.41421356],
       [1.41421356, 1.41421356, 1.41421356, ..., 0.        , 1.41421356,
        0.        ]])

In [17]:
user_similarity_df = pd.DataFrame(user_similarity,index=userRatings.columns,columns=userRatings.columns)
user_similarity_df

title,60 Days In,Akame ga Kill!,Attack on Titan,Avengers: Infinity War,Blade Runner: The Final Cut,Bling Empire,"Bob Ross: Happy Accidents, Betrayal & Greed",Cops and Robbers,Cosmic Sin,Creating an Army of the Dead,...,The Kissing Booth 3,The Secret Life of Pets 2,The Seven Deadly Sins,The Seventh Day,The Show Must Go On: The Queen + Adam Lambert Story,Tinker Bell and the Legend of the NeverBeast,Truth or Dare,Two Distant Strangers,Why Did You Kill Me?,Worth
title,,,,,,,,,,,,,,,,,,,,,
60 Days In,1.000000,-0.111111,-0.111111,-0.166667,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Akame ga Kill!,-0.111111,1.000000,1.000000,-0.166667,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Attack on Titan,-0.111111,1.000000,1.000000,-0.166667,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Avengers: Infinity War,-0.166667,-0.166667,-0.166667,1.000000,-0.166667,-0.166667,-0.166667,-0.166667,0.666667,-0.166667,...,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
Blade Runner: The Final Cut,-0.111111,-0.111111,-0.111111,-0.166667,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Bling Empire,1.000000,-0.111111,-0.111111,-0.166667,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
"Bob Ross: Happy Accidents, Betrayal & Greed",-0.111111,-0.111111,-0.111111,-0.166667,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,1.000000,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111
Cops and Robbers,-0.111111,-0.111111,-0.111111,-0.166667,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,1.000000
Cosmic Sin,-0.111111,-0.111111,-0.111111,0.666667,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111


In [18]:
def get_similar_cosine(movie_name, user_rating):
    similar_score = user_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [20]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_cosine(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

,The Devil Is a Part-Timer!,DOTA: Dragon's Blood,The Seven Deadly Sins,Akame ga Kill!,Durarara!!,Death Note,Record of Ragnarok,The Karate Kid Part II,The Karate Kid,Blade Runner: The Final Cut,...,Spider-Man 3,Rogue Warfare: Death of a Nation,Pieces of a Woman,Ninja Assassin,Creating an Army of the Dead,Cosmic Sin,Cops and Robbers,Worth,Jiu Jitsu,Avengers: Infinity War
0,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,1.666667,-0.277778,-0.277778,-0.277778,...,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.411452,-0.416667
1,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.333333,0.055556,0.055556,0.055556,...,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.082290,0.083333
2,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.250000,-0.166667,-0.166667,-0.166667,...,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.246871,-0.250000
3,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,1.666667,-0.277778,-0.277778,-0.277778,...,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.411452,-0.416667
4,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.000000,-0.166667,-0.166667,-0.166667,...,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.246871,-0.250000


In [21]:
similar_movies.sum().sort_values(ascending=False).head(10)

The Devil Is a Part-Timer!    5.833333
The Seven Deadly Sins         5.833333
Akame ga Kill!                5.833333
Durarara!!                    5.833333
Death Note                    5.833333
DOTA: Dragon's Blood          5.833333
Record of Ragnarok            3.750000
Lucifer                       0.833333
The Kissing Booth 3           0.833333
Friends                       0.833333
dtype: float64

In [15]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(10)

title,60 Days In,Akame ga Kill!,Attack on Titan,Avengers: Infinity War,Blade Runner: The Final Cut,Bling Empire,"Bob Ross: Happy Accidents, Betrayal & Greed",Cops and Robbers,Cosmic Sin,Creating an Army of the Dead,...,The Kissing Booth 3,The Secret Life of Pets 2,The Seven Deadly Sins,The Seventh Day,The Show Must Go On: The Queen + Adam Lambert Story,Tinker Bell and the Legend of the NeverBeast,Truth or Dare,Two Distant Strangers,Why Did You Kill Me?,Worth
title,,,,,,,,,,,,,,,,,,,,,
60 Days In,1.000000,-0.111111,-0.111111,-0.166667,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Akame ga Kill!,-0.111111,1.000000,1.000000,-0.166667,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Attack on Titan,-0.111111,1.000000,1.000000,-0.166667,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Avengers: Infinity War,-0.166667,-0.166667,-0.166667,1.000000,-0.166667,-0.166667,-0.166667,-0.166667,0.666667,-0.166667,...,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
Blade Runner: The Final Cut,-0.111111,-0.111111,-0.111111,-0.166667,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
Bling Empire,1.000000,-0.111111,-0.111111,-0.166667,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111
"Bob Ross: Happy Accidents, Betrayal & Greed",-0.111111,-0.111111,-0.111111,-0.166667,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,1.000000,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111
Cops and Robbers,-0.111111,-0.111111,-0.111111,-0.166667,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,1.000000
Cosmic Sin,-0.111111,-0.111111,-0.111111,0.666667,-0.111111,-0.111111,-0.111111,-0.111111,1.000000,-0.111111,...,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111,-0.111111


In [22]:
def get_similar_pearson(movie_name, user_rating):
    similar_score = corrMatrix[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_score

In [23]:
new_user1 = [("ONE PIECE",5),("Sword Art Online",2),("Sex Education",4),("Hunter X Hunter (2011)",5),("Attack on Titan",4)]
similar_movies = pd.DataFrame()
for movie,rating in new_user1:
    similar_movies = similar_movies.append(get_similar_pearson(movie,rating),ignore_index = True)

for (x,_) in new_user1:
    del similar_movies[x]
    #print("drop "+ x)
similar_movies.head()

,Akame ga Kill!,Durarara!!,Death Note,The Devil Is a Part-Timer!,The Seven Deadly Sins,DOTA: Dragon's Blood,Record of Ragnarok,The Impossible,Tarzan,Friends,...,Creating an Army of the Dead,The Seventh Day,Truth or Dare,Silent Hill: Revelation,The Final Destination,The Conjuring,Scream 2,Seaspiracy,Jiu Jitsu,Avengers: Infinity War
0,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,1.666667,-0.277778,-0.277778,-0.277778,...,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.411452,-0.416667
1,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.500000,-0.333333,0.055556,0.055556,0.055556,...,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.082290,0.083333
2,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.250000,-0.166667,-0.166667,1.500000,...,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.246871,-0.250000
3,2.500000,2.500000,2.500000,2.500000,2.500000,2.500000,1.666667,-0.277778,-0.277778,-0.277778,...,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.277778,-0.411452,-0.416667
4,1.500000,1.500000,1.500000,1.500000,1.500000,1.500000,1.000000,-0.166667,-0.166667,-0.166667,...,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.246871,-0.250000


In [24]:
similar_movies.sum().sort_values(ascending=False).head(10)

Akame ga Kill!                5.833333
The Devil Is a Part-Timer!    5.833333
Durarara!!                    5.833333
Death Note                    5.833333
DOTA: Dragon's Blood          5.833333
The Seven Deadly Sins         5.833333
Record of Ragnarok            3.750000
The Kissing Booth 3           0.833333
Friends                       0.833333
Lucifer                       0.833333
dtype: float64

In [ ]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(20)